In [4]:
import os
import numpy as np
import torch

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import torch.functional as F
import torch.nn as nn

import torch.optim as optim


%matplotlib inline                               


ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (/home/ubuntu/anaconda3/lib/python3.7/site-packages/PIL/__init__.py)

In [2]:
# define trainingm validation and test data directories
data_dir = 'data/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')


In [3]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [5]:
torch.cuda.is_available()

False

In [ ]:
batch_size = 16
workers = 0
data_transform = transforms.Compose([transforms.Resize(512),
                                     transforms.CenterCrop(512),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])
                                    ])

train_data = datasets.ImageFolder(train_dir, data_transform)
valid_data = datasets.ImageFolder(valid_dir, data_transform)
test_data = datasets.ImageFolder(test_dir, data_transform)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=workers, shuffle=True)


In [ ]:
# Visualize some sample data

classes = train_data.classes

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size/2, idx+1, xticks=[], yticks=[])
    image = images[idx]
    image = image.transpose((1, 2, 0))
    image = np.array([0.229, 0.224, 0.225]) * image + np.array([0.485, 0.456, 0.406])
    plt.imshow(image)
    ax.set_title(classes[labels[idx]])

In [ ]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False

In [ ]:
vgg16.classifier

In [ ]:
karims_model = vgg16
karims_model.classifier[-1] = nn.Linear(4096, 3)

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

In [ ]:
epochs = 10 
if train_on_gpu:
    karims_model = karims_model.cuda()
for e in range(1, epochs+1):
    train_loss = 0.
    valid_loss = 0.
    karims_model.train()
    for i, (data, label) in enumerate(train_loader):
        print(".", end="")
        optimizer.zero_grad()
        if train_on_gpu:
            data, label = data.cuda(), label.cuda()
        output = karims_model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step
        train_loss += (loss.item() - train_loss)/(i+1)
        if i % 20 == 1:
            print(f"\n Train Loss: {train_loss}")
        
    karims_model.eval()
    for i, (data, label) in enumerate(valid_loader):
        if train_on_gpu:
            data, label = data.cuda(), label.cuda()
        output = karims_model(data)
        loss = criterion(output, label)
        valid_loss += (loss - valid_loss) / (i+1)
        print("*", end="")
    print(f"\n\n Train loss: {train_loss}\t Validation loss: {valid_loss}")
        

    